In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
api_key = 'cd63a32f7aac7d22fbc21b29f6bd8bd1'

In [3]:
#genre endpoint
url = f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US'
response = requests.get(url)
data = response.json()
genre_dict = {genre['id']: genre['name'] for genre in data['genres']}
print(genre_dict)

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}


In [4]:
#movie discovery endpoint
all_movies = []

for page in range(1, 6):  # Pages 1 to 5
    movie_url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&language=en-US&page={page}'
    response = requests.get(movie_url)
    data = response.json()
    all_movies.extend(data['results'])

In [5]:
df = pd.DataFrame([{
    'Title': movie['title'],
    'Release Date': movie['release_date'],
    'Popularity': movie['popularity'],
    'Vote Average': movie['vote_average'],
    'Vote Count': movie['vote_count'],
    'Genre IDs': movie['genre_ids']
} for movie in all_movies])

print(df.head())

                      Title Release Date  Popularity  Vote Average  \
0  How to Train Your Dragon   2025-06-06   1018.8475         8.100   
1                 M3GAN 2.0   2025-06-25    813.0378         7.760   
2       Karate Kid: Legends   2025-05-08    542.4689         7.272   
3                  Superman   2025-07-09    484.5812         7.456   
4               Almost Cops   2025-07-10    488.6192         5.762   

   Vote Count           Genre IDs  
0         860     [14, 10751, 28]  
1         286       [28, 878, 53]  
2         437        [28, 12, 18]  
3         806       [878, 12, 28]  
4          65  [28, 35, 80, 9648]  


In [6]:
# Convert genre IDs to genre names
df['Genres'] = df['Genre IDs'].apply(lambda ids: [genre_dict.get(i, 'Unknown') for i in ids])

# Convert Release Date column to datetime
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')

cutoff_date = datetime.today() - timedelta(days=90)

# Filtering logic
hidden_gems = df[(df['Vote Average'] >= 7.0) & (df['Vote Count'] < 1000) & (df['Release Date'] < cutoff_date)]
print(hidden_gems)

                           Title Release Date  Popularity  Vote Average  \
25    Kaiju No. 8: Mission Recon   2025-03-28    149.4817         7.955   
26  Jokōsei torio: seikan shiken   1977-02-23    114.6703         7.300   
27               Muromachi Burai   2025-01-17    126.8427         9.222   
32                   The Amateur   2025-04-09     86.2601         7.000   
43                        Belyas   2025-02-07     68.1647         7.350   
51              Stepmom's Desire   2020-05-29     63.8128         7.360   
57   Squid Game: Making Season 2   2025-01-02     51.6217         8.485   
58                         Night   2024-09-07     53.8927        10.000   
69                       Warfare   2025-04-09     47.1837         7.200   
70           The Ugly Stepsister   2025-03-07     46.9958         7.200   
71      Playboy Playmate Workout   1983-01-01     47.8645        10.000   
72                      In Vitro   2025-03-27     74.7202         8.200   
75                       